In [2]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import math

In [3]:
def make_data():
    x = np.random.random(10).reshape(1, -1)
    y = np.zeros((1, 2))
    y[0, np.random.randint(2)] = 1
    return {'features': x, 'label': y}

In [6]:
data = sc.parallelize(range(10000)).map(lambda i: make_data())

In [7]:
data.persist()

PythonRDD[1] at RDD at PythonRDD.scala:48

# Random Split

In [18]:
batch_size = 128
data_cnt = data.count()
batch_ratios = [batch_size / data_cnt] * int(math.ceil(data_cnt / batch_size))

In [16]:
def concate_data(x, y):
    dataset = {}
    for f in ['features', 'label']:
        dataset[f] = np.concatenate((x[f], y[f]), axis=0)
    return dataset


def next_batch_rdd_maker(data_rdd, batch_size, epoch_limit=None):
    data_cnt = data_rdd.count()
    per_batch_ratios = [batch_size / data_cnt] * int(math.ceil(data_cnt / batch_size))
    def next_batch():
        current_epoch = 0
        while True:
            current_epoch += 1
            print('{} epoch starts!'.format(current_epoch))
            if epoch_limit is not None and current_epoch > epoch_limit: 
                break 
            batches = data_rdd.randomSplit(per_batch_ratios)
            for batch in batches:
                dataset = batch.reduce(concate_data)
                yield dataset['features'], dataset['label']
    return next_batch

In [20]:
next_batch = next_batch_rdd_maker(data, batch_size, epoch_limit=2)

i = 0
for X_batch, Y_batch in next_batch():
    print(X_batch.shape, Y_batch.shape)
    print(X_batch[0:5])
    #print(Y_batch[0:5])
    i += 1
    print("run n_batch {}".format(i))

1 epoch starts!
(135, 10) (135, 2)
[[  8.28128172e-01   7.21331148e-01   6.63957775e-02   2.32426736e-01
    4.56041157e-01   8.09835432e-02   7.14077437e-01   8.08210628e-01
    9.86063871e-01   1.42992556e-01]
 [  1.55681195e-01   1.80606717e-01   7.62144989e-01   7.24782388e-01
    8.61966248e-01   5.50625453e-01   3.00610000e-01   1.82758683e-01
    7.76822457e-01   4.62146044e-01]
 [  5.38513563e-01   7.91854510e-01   3.25569815e-01   2.74219205e-01
    1.12040933e-01   8.72800603e-01   7.03616396e-01   6.40697977e-01
    9.67057467e-02   8.39538563e-01]
 [  9.08446212e-02   4.05568746e-01   6.85400665e-01   1.75869853e-01
    1.93149088e-01   5.93925187e-02   2.21232991e-04   7.83807873e-01
    3.67174324e-01   9.03314647e-01]
 [  4.24758321e-01   4.64913288e-01   6.33092228e-01   5.23019306e-01
    1.02766232e-01   7.57264995e-02   9.73231216e-01   1.42144926e-02
    8.92033913e-01   1.07064273e-01]]
run n_batch 1
(101, 10) (101, 2)
[[ 0.4436542   0.32137206  0.507971    0.68325

# Group

In [ ]:
#batch_rdd = train_data.sortBy(lambda x: np.random.random()).zipWithIndex().map(lambda x: (x[1]%10, x[0])).groupByKey()
for i, batch in batch_rdd.collect():
    print(len(batch))

# MapPartition

In [21]:
data.take(1)

[{u'features': array([[ 0.54347224,  0.97957591,  0.43218624,  0.98354428,  0.18247286,
           0.4947466 ,  0.54333409,  0.1123695 ,  0.72597541,  0.61974174]]),
  u'label': array([[ 0.,  1.]])}]

In [ ]:
data_rdd

In [7]:
data_count = data_rdd.count()
batch_size = 50
n_batch = data_count // batch_size

In [8]:
data_count, n_batch

(100000, 2000)

In [27]:
batch_rdd = data_rdd \
    .coalesce(n_batch, shuffle=True) \
    .zipWithIndex() \
    .map(lambda x: (x[1]%n_batch , x[0])) \
    .groupByKey() \
    .map(lambda x: x[1])

In [28]:
for x in batch_rdd.take(1)[0]:
    print(x)

{u'features': array([[ 0.37073191,  0.48244528,  0.28461215,  0.1317227 ,  0.87890136,
         0.01102265,  0.7651571 ,  0.25752815,  0.75159029,  0.64502424]]), u'label': array([[ 0.,  1.]])}
{u'features': array([[ 0.4504058 ,  0.66501192,  0.2962715 ,  0.02181195,  0.65113811,
         0.90853561,  0.097354  ,  0.58486196,  0.18357662,  0.76886389]]), u'label': array([[ 1.,  0.]])}
{u'features': array([[ 0.95342291,  0.4535336 ,  0.35981249,  0.50075611,  0.99902678,
         0.6014667 ,  0.53381058,  0.48165125,  0.92430657,  0.05714938]]), u'label': array([[ 0.,  1.]])}
{u'features': array([[ 0.52305435,  0.41638638,  0.71556628,  0.70230437,  0.09229949,
         0.92923604,  0.30086101,  0.4653124 ,  0.64653215,  0.21608639]]), u'label': array([[ 1.,  0.]])}
{u'features': array([[ 0.01223647,  0.24719584,  0.86281775,  0.45068686,  0.07528193,
         0.89202074,  0.76245591,  0.34326625,  0.34497815,  0.65970768]]), u'label': array([[ 0.,  1.]])}
{u'features': array([[ 0.33977

In [24]:
batch_rdd.getNumPartitions()

2000

In [31]:
batch_rdd.take(1)

[{u'features': array([[ 0.93172863,  0.98480599,  0.95741531,  0.6129436 ,  0.19673222,
           0.05387264,  0.63988158,  0.11117587,  0.54039591,  0.04718286]]),
  u'label': array([[ 0.,  1.]])}]

In [36]:
def concat_data(it):
    dataset = None
    for d in it:
        if dataset is None:
            dataset = d
        else:
            for f in ['features', 'label']:
                dataset[f] = np.concatenate((dataset[f], d[f]), axis=0)
    yield dataset

batch_rdd.mapPartitions(concat_data).filter(lambda x: x is not None).collect()

[{u'features': array([[ 0.93172863,  0.98480599,  0.95741531,  0.6129436 ,  0.19673222,
           0.05387264,  0.63988158,  0.11117587,  0.54039591,  0.04718286]]),
  u'label': array([[ 0.,  1.]])},
 {u'features': array([[ 0.31638226,  0.85218441,  0.55351473,  0.15053392,  0.09371248,
           0.20815976,  0.86655822,  0.61954134,  0.37148639,  0.92871957],
         [ 0.01892403,  0.39535147,  0.0542648 ,  0.37229283,  0.51681396,
           0.19004711,  0.27305341,  0.32618019,  0.02619873,  0.14676534]]),
  u'label': array([[ 1.,  0.],
         [ 1.,  0.]])},
 {u'features': array([[  3.20721424e-01,   8.64023243e-01,   8.06594026e-01,
            7.97330553e-01,   7.52700299e-01,   7.73324399e-01,
            4.98287828e-01,   9.94801137e-01,   9.96904925e-01,
            2.22312982e-01],
         [  6.77889180e-01,   1.70671847e-01,   6.40587954e-01,
            3.13694353e-01,   5.06828399e-01,   4.71038664e-01,
            4.23924298e-01,   7.47247740e-01,   1.80615691e-01,
  